<a href="https://colab.research.google.com/github/GouriGiddale26/diabetes/blob/main/diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes (1).csv


In [16]:
import numpy as np
import pandas as pd

df = pd.read_csv("diabetes.csv")

In [17]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
df.shape

(768, 9)

In [19]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [21]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [22]:
(df == 0).any()

,0
Pregnancies,True
Glucose,True
BloodPressure,True
SkinThickness,True
Insulin,True
BMI,True
DiabetesPedigreeFunction,False
Age,False
Outcome,True


In [23]:
df.columns = df.columns.str.strip().str.lower()

In [24]:
# columns whrer 0 is not valid medical value
invalid_zero_cols = [
    'glucose',
    'bloodpressure',
    'skinthickness',
    'insulin',
    'bmi'
]
# replace 0 with nan
df[invalid_zero_cols] = df[invalid_zero_cols].replace(0, np.nan)

In [25]:
# impute missing values

for col in invalid_zero_cols:
  df[col].fillna(df[col].median(),inplace=True)

/tmp/ipython-input-1773109559.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(),inplace=True)


In [26]:
df.isnull().sum()

,0
pregnancies,0
glucose,0
bloodpressure,0
skinthickness,0
insulin,0
bmi,0
diabetespedigreefunction,0
age,0
outcome,0


In [27]:
# feature target separation
# feature
x =df.drop(columns =['outcome'])
y = df['outcome']

In [28]:
x.columns

Index(['pregnancies', 'glucose', 'bloodpressure', 'skinthickness', 'insulin',
       'bmi', 'diabetespedigreefunction', 'age'],
      dtype='object')

In [29]:
y.name

'outcome'

In [30]:
# train and test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(
    x,y,test_size=0.2,random_state=42
)

In [31]:
x_train.shape,x_test.shape

((614, 8), (154, 8))

In [32]:
y.value_counts()

,count
outcome,
0,500
1,268


In [33]:
y_test.value_counts()

,count
outcome,
0,99
1,55


In [34]:
y_test.value_counts(normalize=True)

,proportion
outcome,
0,0.642857
1,0.357143


In [35]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [36]:
# here using logistic regression classifier because it is a binary classification problem, sstrong baseline for medical data

from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(x_train_scaled,y_train)

LogisticRegression()

In [37]:
y_pred = log_reg.predict(x_test_scaled)

In [38]:
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [39]:
log_reg.score(x_test_scaled,y_test)

0.7532467532467533

In [40]:
log_reg.score(x_train_scaled,y_train)

0.7703583061889251

In [41]:
y_prob = log_reg.predict_proba(x_test_scaled)

In [42]:
y_prob[:20]

array([[0.72650648, 0.27349352],
       [0.83274552, 0.16725448],
       [0.89278831, 0.10721169],
       [0.84589992, 0.15410008],
       [0.53748186, 0.46251814],
       [0.56562556, 0.43437444],
       [0.98601508, 0.01398492],
       [0.6160686 , 0.3839314 ],
       [0.4244803 , 0.5755197 ],
       [0.23804933, 0.76195067],
       [0.77764247, 0.22235753],
       [0.10709486, 0.89290514],
       [0.62464797, 0.37535203],
       [0.72006814, 0.27993186],
       [0.92184111, 0.07815889],
       [0.59630803, 0.40369197],
       [0.88761243, 0.11238757],
       [0.92622869, 0.07377131],
       [0.27479951, 0.72520049],
       [0.42204828, 0.57795172]])

In [43]:
y_prob_diabetic = y_prob[:,1]
y_prob_diabetic[:10]

array([0.27349352, 0.16725448, 0.10721169, 0.15410008, 0.46251814,
       0.43437444, 0.01398492, 0.3839314 , 0.5755197 , 0.76195067])

In [44]:
# confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm
# [TN FP]
# [FN TP]

# TN- true negative
# FP- false positive
# FN- false negative
# TP- true positive

array([[82, 17],
       [21, 34]])

In [45]:
# classification report

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

# here we will see about precision, recall, f1 score and support
# precision: how correct the model was when it said 0 or 1
# recall: how many real 0 and 1 it found
# f1:balance score of precision and recall
# support : how many actual 0 and 1 patients
#exmaple for weighted avg
#classes with more samples influence the score more than the smaller classes

              precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154



In [46]:
#next model we using here is SVM support vector
#machine it finds a strong margin based decision
#boundary that otean imporv well scalled datasets
#the  main reason to use is dateset is small in size,
#we did the scaling,chances are high to get better recall


In [47]:
from sklearn.svm import SVC
svm_model = SVC(kernel ='linear', probability=True)
svm_model.fit(x_train_scaled,y_train)

SVC(kernel='linear', probability=True)

In [48]:
y_pred_svm = svm_model.predict(x_test_scaled)
svm_model.score(x_test_scaled,y_test)

0.7532467532467533

In [49]:
y_prob_svm = svm_model.predict_proba(x_test_scaled)[:,1]

In [50]:
y_prob_svm[:10]

array([0.29684281, 0.18548366, 0.12975705, 0.17502688, 0.4618714 ,
       0.45002776, 0.02568032, 0.37050703, 0.5450962 , 0.72955004])

In [51]:
# confusion matrix

from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,y_pred_svm))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154



In [52]:
#confusion matrics

from sklearn.metrics import confusion_matrix,classification_report
cm_svm = confusion_matrix(y_test,y_pred_svm)
cm_svm

array([[82, 17],
       [21, 34]])

In [53]:
# ROC AUC Receiver Operating Characteristic, Area Under the Curve

from sklearn.metrics import roc_auc_score
roc_auc_lr = roc_auc_score(y_test,y_prob[:,1])
roc_auc_svm = roc_auc_score(y_test,y_prob_svm)

roc_auc_lr,roc_auc_svm

(np.float64(0.82277318640955), np.float64(0.8216712580348944))